<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [ ]:
!pip3 install folium
!pip3 install wget
!pip3 install pandas

In [2]:
import folium
import wget
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [4]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first() #Otra forma de dejar solo un elemento de cada tipo
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [6]:
spacex_df.head()

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS LC-40,28.562302,-80.577356,0
2,CCAFS LC-40,28.562302,-80.577356,0
3,CCAFS LC-40,28.562302,-80.577356,0
4,CCAFS LC-40,28.562302,-80.577356,0


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [9]:
launch_sites_df.dtypes

Launch Site     object
Lat            float64
Long           float64
dtype: object

In [10]:
#Arreglado eran los {} del site
# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Loop through each launch site to add Circle and Marker
for _, site in launch_sites_df.iterrows():  # `_` ignores the index as it's not used
    # Coordinates for the site
    site_coordinate = (site['Lat'], site['Long']) # tambien vale tuple([site['Lat'], site['Long']])
    
    # Add a Circle for the site with popup label
    circle = folium.Circle(
        location=site_coordinate,
        radius=1000,
        color='#d35400',
        fill=True,
        fill_opacity=0.05
    ).add_child(folium.Popup(site['Launch Site']))  # Popup with launch site name
    
    # Add a Marker with the site name as a label
    marker = folium.Marker(
        location=site_coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{site["Launch Site"]}</b></div>' #No olvidar los {}
        )
    )
    
    # Add both the Circle and the Marker to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map
site_map
# Las etiquetas salen muy juntas, pero aun poniendo un offset, están tan juntas que no queda bien

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?  
Tanto como han podido sin salir de los usa.
- Are all launch sites in very close proximity to the coast?  
Si, todas, o a una gran extensión de agua.

Also please try to explain your findings.  
Contra mas cercano al ecuador es mas facil y productivo lanzar los objetos al espacio y contra mas cerca del mar, menos gente a la que poner en peligro

# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [11]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [12]:
marker_cluster = MarkerCluster()


_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


No tiene sentido crearla en lauch_sites, se va a utilizar spacex_df, ya que se marca cada uno de los lanzamientos, no a la site de lanzamiento

In [13]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red


In [14]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [15]:
# Add marker_cluster to current site_map, que magicamente se acuerda
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # Coordinates for the site
    site_coordinate = (record['Lat'], record['Long']) # tambien vale tuple([site['Lat'], site['Long']])
    marker = folium.Marker(
        location=site_coordinate,
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [16]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [17]:
site_map.save("my_map.html")

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Costa only 2, the other one is a ¿channel?  
(28.56356, -80.56785)  
(23.63266, -120.6266)  


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [18]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [19]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
# No need to check all launch sites from all coastlines, but..
# Coastline coordinates list
coastline_coords = [
    (28.56356, -80.56785)
    , (23.63266, -120.6266)
]

# Initialize an empty list to store results
distance_results = []

# Loop through each launch site in spacex_df
for _, launch_site in spacex_df.iterrows(): # No olvidar el indice aunque no se utilice
    launch_lat, launch_lon = launch_site['Lat'], launch_site['Long'] # el metodo lo pide por separado, no tuplas como antes
    
    # No hace falta calcularlas todas, pero como solo son 2x4...
    for coast_lat, coast_lon in coastline_coords:
        distance = calculate_distance(launch_lat, launch_lon, coast_lat, coast_lon)
        distance_results.append({
            'Launch Site': launch_site['Launch Site'],
            'Launch Site Coords': (launch_lat, launch_lon),
            'Coastline Coords': (coast_lat, coast_lon),
            'Distance (km)': distance
        })

# Convert results into a DataFrame
distance_df = pd.DataFrame(distance_results)

# Find the minimum distance for each launch site
min_distances = distance_df.loc[distance_df.groupby('Launch Site')['Distance (km)'].idxmin()]

min_distance = min_distances.loc[min_distances['Distance (km)'].idxmin()]

min_distances

,Launch Site,Launch Site Coords,Coastline Coords,Distance (km)
0,CCAFS LC-40,"(28.56230197, -80.57735648)","(28.56356, -80.56785)",0.939193
98,CCAFS SLC-40,"(28.56319718, -80.57682003)","(28.56356, -80.56785)",0.877228
72,KSC LC-39A,"(28.57325457, -80.64689529)","(28.56356, -80.56785)",7.796646
53,VAFB SLC-4E,"(34.63283416, -120.6107455)","(23.63266, -120.6266)",1223.548501


_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [20]:
min_distance

Launch Site                          CCAFS SLC-40
Launch Site Coords    (28.56319718, -80.57682003)
Coastline Coords            (28.56356, -80.56785)
Distance (km)                            0.877228
Name: 98, dtype: object

In [21]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
coordinate = (min_distance['Coastline Coords'][0], min_distance['Coastline Coords'][1])
distance = min_distance['Distance (km)']
distance_marker = folium.Marker(
   coordinate,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       #html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
       html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance),
       )
   )
    # marker = folium.Marker(...)
marker_cluster.add_child(distance_marker)
site_map

_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [22]:
min_distance

Launch Site                          CCAFS SLC-40
Launch Site Coords    (28.56319718, -80.57682003)
Coastline Coords            (28.56356, -80.56785)
Distance (km)                            0.877228
Name: 98, dtype: object

In [23]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
coordinates = [min_distance['Launch Site Coords'],min_distance['Coastline Coords']]
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


In [24]:
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


city:       28.40650 -80.60429  
high way:   28.57336 -80.65538  
railway:    34.63310 -120.62488  

A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
def calculate_distances(origen_df, destination_coords, calculate_distance):
    """
    Calcula las distancias entre las listas de las coordenadas de los sitios de lanzamiento y de las coordenadas de destino.
    
    Parameters:
    origen_df (pd.DataFrame): DataFrame que contiene los sitios de lanzamiento.
    destination_coords (list of tuples): Lista de tuplas con las coordenadas (latitud, longitud) de los puntos a calcular distancias.
    calculate_distance (function): Función que calcula la distancia entre dos puntos geográficos.
    
    Returns:
    list of dict: Lista de diccionarios con los resultados de las distancias.
    """
    distance_results = []

    # Loop through each launch site in spacex_df
    for _, launch_site in origen_df.iterrows():
        launch_lat, launch_lon = launch_site['Lat'], launch_site['Long']
        
        # Loop through each destination (city) and calculate the distance
        for dest_lat, dest_lon in destination_coords:
            distance = calculate_distance(launch_lat, launch_lon, dest_lat, dest_lon)
            distance_results.append({
                'Launch Site': launch_site['Launch Site'],
                'Launch Site Coords': (launch_lat, launch_lon),
                'Destination Coords': (dest_lat, dest_lon),
                'Distance (km)': distance
            })
    
    return distance_results


In [26]:
# Just one set of coordinates, but ready for many of them
# city_coords = [
#     (28.56356, -80.56785)
#     , (23.63266, -120.6266)
# ]
# Lista de coordenadas de ciudades (solo una ciudad en este caso)
city_coords = [
    (28.4065, -80.60429)
]

# En estos casos solo ponemos un set de coordenadas destino, pero podriamos poner un grupo de ellos sin problema
# Inicializar la lista de resultados de distancias
distance_results = []

# Calcular las distancias
distance_results = calculate_distances(spacex_df, city_coords, calculate_distance)

# Convertir los resultados a un DataFrame
distance_df = pd.DataFrame(distance_results)

# Encontrar la distancia mínima para cada sitio de lanzamiento
min_distances = distance_df.loc[distance_df.groupby('Launch Site')['Distance (km)'].idxmin()]

# Encontrar la distancia mínima global
min_distance = min_distances.loc[min_distances['Distance (km)'].idxmin()]

# Extraer las coordenadas del sitio de lanzamiento y la coordenada de destino
launch_coords = min_distance['Launch Site Coords']
destination_coords = min_distance['Destination Coords']

# Desempaquetar las coordenadas de latitud y longitud
launch_lat, launch_lon = launch_coords
destination_lat, destination_lon = destination_coords

# Crear una lista de coordenadas para la PolyLine
coordinates = [(launch_lat, launch_lon), (destination_lat, destination_lon)]

# Crear el marcador con la distancia
distance = min_distance['Distance (km)']
distance_marker = folium.Marker(
    [destination_lat, destination_lon],  # Coordenadas del destino
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance),  # Correcto uso de .format
    )
)

# Añadir el marcador al mapa
marker_cluster.add_child(distance_marker)

# Crear un `folium.PolyLine` usando las coordenadas del sitio de lanzamiento y el destino
lines = folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

print("Aproximation to city")
print(min_distance)
# Mostrar el mapa
site_map


Aproximation to city
Launch Site                           CCAFS LC-40
Launch Site Coords    (28.56230197, -80.57735648)
Destination Coords           (28.4065, -80.60429)
Distance (km)                           17.528731
Name: 0, dtype: object


In [27]:
# Lista de coordenadas de xxx
highway_coords = [
    (28.57336, -80.65538)
]

# En estos casos solo ponemos un set de coordenadas destino, pero podriamos poner un grupo de ellos sin problema
# Inicializar la lista de resultados de distancias
distance_results = []

# Calcular las distancias
distance_results = calculate_distances(spacex_df, highway_coords, calculate_distance)

# Convertir los resultados a un DataFrame
distance_df = pd.DataFrame(distance_results)

# Encontrar la distancia mínima para cada sitio de lanzamiento
min_distances = distance_df.loc[distance_df.groupby('Launch Site')['Distance (km)'].idxmin()]

# Encontrar la distancia mínima global
min_distance = min_distances.loc[min_distances['Distance (km)'].idxmin()]

# Extraer las coordenadas del sitio de lanzamiento y la coordenada de destino
launch_coords = min_distance['Launch Site Coords']
destination_coords = min_distance['Destination Coords']

# Desempaquetar las coordenadas de latitud y longitud
launch_lat, launch_lon = launch_coords
destination_lat, destination_lon = destination_coords

# Crear una lista de coordenadas para la PolyLine
coordinates = [(launch_lat, launch_lon), (destination_lat, destination_lon)]

# Crear el marcador con la distancia
distance = min_distance['Distance (km)']
distance_marker = folium.Marker(
    [destination_lat, destination_lon],  # Coordenadas del destino
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance),  # Correcto uso de .format
    )
)

# Añadir el marcador al mapa
marker_cluster.add_child(distance_marker)

# Crear un `folium.PolyLine` usando las coordenadas del sitio de lanzamiento y el destino
lines = folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

print("Aproximation to highway")
print(min_distance)
# Mostrar el mapa
site_map


Aproximation to highway
Launch Site                            KSC LC-39A
Launch Site Coords    (28.57325457, -80.64689529)
Destination Coords          (28.57336, -80.65538)
Distance (km)                            0.828892
Name: 36, dtype: object


In [28]:
# Lista de coordenadas de xxx
railway_coords = [
    (34.6331, -120.62488)
]

# En estos casos solo ponemos un set de coordenadas destino, pero podriamos poner un grupo de ellos sin problema
# Inicializar la lista de resultados de distancias
distance_results = []

# Calcular las distancias
distance_results = calculate_distances(spacex_df, railway_coords, calculate_distance)

# Convertir los resultados a un DataFrame
distance_df = pd.DataFrame(distance_results)

# Encontrar la distancia mínima para cada sitio de lanzamiento
min_distances = distance_df.loc[distance_df.groupby('Launch Site')['Distance (km)'].idxmin()]

# Encontrar la distancia mínima global
min_distance = min_distances.loc[min_distances['Distance (km)'].idxmin()]

# Extraer las coordenadas del sitio de lanzamiento y la coordenada de destino
launch_coords = min_distance['Launch Site Coords']
destination_coords = min_distance['Destination Coords']

# Desempaquetar las coordenadas de latitud y longitud
launch_lat, launch_lon = launch_coords
destination_lat, destination_lon = destination_coords

# Crear una lista de coordenadas para la PolyLine
coordinates = [(launch_lat, launch_lon), (destination_lat, destination_lon)]

# Crear el marcador con la distancia
distance = min_distance['Distance (km)']
distance_marker = folium.Marker(
    [destination_lat, destination_lon],  # Coordenadas del destino
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance),  # Correcto uso de .format
    )
)

# Añadir el marcador al mapa
marker_cluster.add_child(distance_marker)

# Crear un `folium.PolyLine` usando las coordenadas del sitio de lanzamiento y el destino
lines = folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

print("Aproximation to railway")
print(min_distance)
# Mostrar el mapa
site_map

Aproximation to railway
Launch Site                           VAFB SLC-4E
Launch Site Coords    (34.63283416, -120.6107455)
Destination Coords          (34.6331, -120.62488)
Distance (km)                            1.293941
Name: 26, dtype: object


After you plot distance lines to the proximities, you can answer the following questions easily:  
First should know what is considered close proximity.

- Are launch sites in close proximity to railways?
There are some railways close.
- Are launch sites in close proximity to highways?
Yes, as railways.
- Are launch sites in close proximity to coastline?
Yes, they try to be, so it is safer.
- Do launch sites keep certain distance away from cities?
Aa much as they can for avoiding potential dangers to general population.

Also please try to explain your findings.
Moving spaceship parts, specially heavy parts require big transport, they need railways and highways for moving all these parts and cargo, the 
most reusable big parts they can get, the less they will need transporting big loads of components, and usually cargo is not so big so it doesn't 
need so special transports and the spaceship components do.  
And relative to the cities, yes, they avoid city centers as a possible colateral damage if it all goes wrong.

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
